<a href="https://colab.research.google.com/github/gsiolas/nn-notebooks/blob/main/07_Data_Preprocessing_with_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preprocessing with `pandas`

We'll work on a dataset for credit scoring that was proposed some years ago as a data challenge on some data challenge website.
It is a realistic dataset that contains a lot of missing values, several types of features (dates, categories, continuous features), so that a serious data cleaning and formating is required.

# Importing the data

We start by downloading the data. You need to do this only **once** since the next cell downloads the `gro` dataset. This dataset contains the following columns:

| Column name          | Description |
|:---------------------|:------------|
| BirthDate            | Date de naissance |
| Customer_Open_Date   | Date d'arrivée du client dans la filiale bancaire |
| Customer_Type        | Type de client (existant / nouveau) | 
| Educational_Level    | Niveau de diplôme |
| Id_Customer          | Identifiant du client |
| Marital_Status       | Situation familiale |
| Nb_Of_Products       | Nombre de produits détenus par le client à l'octroi |
| Net_Annual_Income    | Revenu annuel |
| Number_Of_Dependant  | Nombre de personnes à charge |
| P_Client             | Distinction des clients selon une caractéristique non définie ici |
| Prod_Category        | Catégorie du produit |
| Prod_Closed_Date     | Date de fermeture du produit |
| Prod_Decision_Date   | Date de décision de l'octroi (du financement) |
| Prod_Sub_Category    | Sous-catégorie du produit |
| Source               | Source de financement (Branch or Sales) |
| Type_Of_Residence    | Situation résidentielle |
| Y                    | Crédit accordé ou non | |
| Years_At_Business    | Nombre d'années dans son emploi actuel |
| Years_At_Residence   | Nombre d'années dans son lieu de résidence actuel |

In [ ]:
import requests
import os

# The path containing your notebook
path_data = './'
# The name of the file
filename = 'gro.csv.gz'

if os.path.exists(os.path.join(path_data, filename)):
    print('The file %s already exists.' % os.path.join(path_data, filename))
else:
    url = 'http://stephanegaiffas.github.io/files/m2mo/gro.csv.gz'
    r = requests.get(url)
    with open(os.path.join(path_data, filename), 'wb') as f:
        f.write(r.content)
    print('Downloaded file %s.' % os.path.join(path_data, filename))

## A quick and easy (but actually bad) import

Let's import the data into a pandas dataframe, as simply as possible
The only thing we care about for now is the fact that the column separator 
is `';'` and not `','` as it should be in a `.csv` file.

In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv(os.path.join(path_data, filename), sep=';')

In [ ]:
type(data)

In [ ]:
# Let's look at the first lines of the dataframe
data.head(n=5)

In [ ]:
# The list of column names
data.columns

**Remark**. There are weird columns in the end, they look empty. 
They don't appear in the description of the data.

In [ ]:
# Let's look at the types used for each column
data.dtypes

In [ ]:
data.BirthDate.head()

In [ ]:
# Birth date is imported as a string (all columns containing dates do)
type(data['BirthDate'][0])

In [ ]:
data['Prod_Sub_Category'].head()

In [ ]:
# Product sub category is imported as a string
type(data['Prod_Sub_Category'][0])

In [ ]:
data['Net_Annual_Income'].head(n=20)

In [ ]:
# Net actual income is a string as well ! While it is clearly a number
type(data['Net_Annual_Income'][0])

In [ ]:
# Quick summary statistics of the dataset features'
data.describe(include='all')

## Let's assess what we did

It appears that we have to work a little bit more. 
Here is the list of problems we have.
- The last three columns are weird and empty
- Dates are actually `str` (python's string type)
- There is a lot of missing values
- Categorial features are `str`
- The annual net income is imported as a string

By looking at the column names, basic statistics and the dataset 
description, we infer the type of features that we have.
There are dates features, continuous features, categorical features, and
some features could be either treated as categorical or continuous.

There is **a lot** of missing values, that need to be handled somehow.

The annual net income is imported as a string, we need to understand why.

We really need to treat dates as dates and not string (because we'd want to compute the age of a client based on its birth year for instance).

Here is the structure of the features.

**Continuous features**
- Years_At_Residence
- Net_Annual_Income
- Years_At_Business

**To decide features**
- Number_Of_Dependant
- Nb_Of_Products

**Categorical features are**
- Customer_Type
- P_Client
- Educational_Level
- Marital_Status
- Prod_Sub_Category
- Source
- Type_Of_Residence
- Prod_Category

**Date features are**
- BirthDate
- Customer_Open_Date
- Prod_Decision_Date
- Prod_Closed_Date

# A closer look at the import problems

Let's find solutions to all these import problems.

## The last three columns are weird and empty 

It seems to come from the fact that the data always ends with several `';'` characters. Let's simply remove them using the `usecols`
option.

## Dates are actually `str` (python's string type)

We need to specify which columns must be encoded as dates using the `parse_dates` option. Fortunately enough, the `pandas` is clever enough to interpret the date format.

In [ ]:
type(data['BirthDate'][0])

## There is a lot of missing values 

We'll see below that actually a single column mostly contain missing values.

In [ ]:
data.isnull().sum()

In [ ]:
# The "product closed date" seems to contain a lot of missing values !
data[['Prod_Closed_Date']].head(5)

In [ ]:
# We remove the useless columns from the dataframe
data = data.drop(['Prod_Closed_Date', 'Unnamed: 19', 
                            'Unnamed: 20', 'Unnamed: 21'], axis=1)

In [ ]:
data.head()

In [ ]:
# The remaining lines with missing values
data[data.isnull().any(axis=1)]

## Categorial features are `str`

We need to say the dtype we want to use for some columns using the `dtype` option.

In [ ]:
# Product sub category is imported as a string
type(data['Prod_Sub_Category'][0])

In [ ]:
data['Prod_Sub_Category'].unique()

## The annual net income is imported as a string

This problem comes from the fact that the decimal separator is in European notation: it's a `','` and not a `'.'`, so we need to specify it using the `decimal` option.

In [ ]:
type(data['Net_Annual_Income'][0])

In [ ]:
data['Net_Annual_Income'].head(n=10)

# A better import of the data

## Let's import again the data

- We build a dict that specifies the types to use for each column 
and pass it to `read_csv` using the `dtype` option
- We also specify the `decimal`, `usecols` and `parse_dates` options.

**Important remark (expert)**. Some columns could be `np.int`. 
However, `pandas` (actually its `numpy`) does not support columns 
with integer dtype and missing values.

In [ ]:
gro_dtypes = {
    'Years_At_Residence': np.int,
    'Net_Annual_Income' : np.float,
    'Years_At_Business': np.float,
    'Number_Of_Dependant': np.float,
    'Nb_Of_Products': np.int,
    'Customer_Type': 'category',
    'P_Client': 'category',
    'Educational_Level': 'category',
    'Marital_Status': 'category',
    'Prod_Sub_Category': 'category',
    'Source': 'category',
    'Type_Of_Residence': 'category',
    'Prod_Category': 'category',
}

data = pd.read_csv(os.path.join(path_data, filename), 
                   sep=';', decimal=',', usecols=range(19),
                   parse_dates=['BirthDate', 'Customer_Open_Date', 
                                'Prod_Decision_Date', 'Prod_Closed_Date'],
                   dtype=gro_dtypes)

In [ ]:
data.dtypes

In [ ]:
# Product subcategory is imported as a string as well
data['Prod_Sub_Category'].head()

In [ ]:
data['BirthDate'].head()

In [ ]:
data['BirthDate'][0]

In [ ]:
# So let's simply remove this column for now
prod_closed_date = data.pop('Prod_Closed_Date')
data.shape

In [ ]:
# And remove the remaining rows with missing values
data = data.dropna()
data.shape

In [ ]:
# Now we save the cleaned dataset it into a CSV file
data.to_csv(os.path.join(path_data, 'gro_cleaned.csv'))

## The net income columns is very weird

In [ ]:
data.columns

In [ ]:
income = data['Net_Annual_Income']
income.describe()

The net income has a very weird distribution: a non-negligible number of very large values.

In [ ]:
(income <= 100).sum(), (income > 100).sum()

Most values are smaller than 100, while some are much larger...

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_context("notebook", font_scale=1.2)

In [ ]:
plt.figure(figsize=(10, 6))
_ = plt.hist(income, bins=10)

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(income[income <= 1000])
plt.subplot(1, 2, 2)
plt.hist(income[income > 1000])
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8, 5))
sns.distplot(income[income < 300])

Seems that there's a bug in the net income features. 
Some numbers are in kilo euros, while others are in euros.
Let's clean this by removing the rows with an income larger than 200.

In [ ]:
plt.figure(figsize=(8, 5))
sns.distplot(income[income < 200])

In [ ]:
data = data[income <= 200]

# Some data visualization with `pandas` + `seaborn`

In [ ]:
plt.figure(figsize=(8, 5))
sns.stripplot(x='Educational_Level', y='Net_Annual_Income', hue='Y', 
              jitter=True, data=data)

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='Educational_Level', y='Net_Annual_Income', 
            hue='Y', data=data)

In [ ]:
plt.figure(figsize=(12, 6))
sns.violinplot(x='Marital_Status', y='Net_Annual_Income', 
               hue='Y', split=True, data=data)

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(x='Marital_Status', hue='Y', data=data)

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(16, 16))
columns = ['Customer_Type', 'P_Client', 'Educational_Level', 
           'Number_Of_Dependant', 'Marital_Status', 'Prod_Sub_Category',
           'Source', 'Type_Of_Residence', 'Nb_Of_Products', 
           'Prod_Category', 'Y']

for i, colname in enumerate(columns):
    sns.countplot(colname, data=data, ax=fig.axes[i])
plt.tight_layout()

# Final preparation of the dataset

In [ ]:
# First we make lists of continuous, categorial and date features

cnt_featnames = [
    'Years_At_Residence',
    'Net_Annual_Income',
    'Years_At_Business',
    'Number_Of_Dependant'
]

cat_featnames = [
    'Customer_Type',
    'P_Client',
    'Educational_Level',
    'Marital_Status',
    'Prod_Sub_Category',
    'Source',
    'Type_Of_Residence',
    'Prod_Category',
    'Nb_Of_Products'
]

date_featnames = [
    'BirthDate',
    'Customer_Open_Date',
    'Prod_Decision_Date'
#    'Prod_Closed_Date'
]

## Creation of the features matrix

In [ ]:
data[cnt_featnames].head()

In [ ]:
bin_features = pd.get_dummies(data[cat_featnames],
                              prefix_sep='#', drop_first=True)

In [ ]:
bin_features.head()

In [ ]:
cnt_features = data[cnt_featnames]
cnt_features.head()

In [ ]:
from pandas import Timestamp

def age(x):
    today = Timestamp.today()
    return (today - x).dt.days

date_features = data[date_featnames].apply(age, axis=0)
date_features.head()

## Final features matrix

In [ ]:
all_features = pd.concat([bin_features, cnt_features, date_features], axis=1)

In [ ]:
all_features.columns

In [ ]:
all_features.head()

**VERY IMPORTANT**: we removed lines of data that contained missing values. The index of the dataframe is
    therefore not contiguous anymore

In [ ]:
all_features.index.max()

This could be a problem for later. So let's reset the index to get a contiguous one

In [ ]:
all_features.shape

In [ ]:
all_features.reset_index(inplace=True, drop=True)

In [ ]:
all_features.head()

## Let's save the data using `pickle`

In [ ]:
import pickle as pkl

X = all_features
y = data['Y']

data_pkl = {}
data_pkl['features'] = X
data_pkl['labels'] = y

data_pkl['cnt_featnames'] = cnt_featnames
data_pkl['cat_featnames'] = cat_featnames
data_pkl['date_featnames'] = date_featnames

with open(os.path.join(path_data, 'gro_training.pkl'), 'wb') as f:
    pkl.dump(data_pkl, f)

The preprocessed data is saved in a pickle file called `gro_training.pkl`.

## Another approach using `scikit-learn`

![](https://miro.medium.com/max/1050/0*j9TC7PTOq1YaQ5se.png)

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# First we create a scikit-learn encoder that computes 
#  the age in days of columns containing dates
class AgeEncoder(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        self.today = Timestamp.today()
        return self

    def transform(self, X):
        return X.apply(lambda x: (x - self.today).dt.days, axis=0)

# Centers and reduces (variance=1) columns
standard_scaler = StandardScaler()

# One-hot encode, similar to pd.get_dummies
one_hot_encoder = OneHotEncoder(drop='first', sparse=False)

# A pipeline that first computes age, and standardizes it
scaled_age_encoder = Pipeline([
    ('age', AgeEncoder()),
    ('scaling', StandardScaler())
])

# Let's combine all these transformations
transformer = ColumnTransformer([
    ('standard_scaling', standard_scaler, cnt_featnames),
    ('one_hot_encoding', one_hot_encoder, cat_featnames),
    ('dates_age_scaled', scaled_age_encoder, date_featnames)
])

transformer.fit_transform(data)